<a href="https://colab.research.google.com/github/GoldenWanGila/Collage-Project/blob/master/DEF%E8%87%AA%E5%BC%B7%E5%8C%97%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
from re import T
import sklearn.linear_model
import seaborn as sns; sns.set()
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sympy import *
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import matplotlib as mpl
import pandas as pd
import numpy as np
import datetime

In [187]:
def plot(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    print(sqrt(v))

In [188]:
def arrangeDataFrame(x_dataFrame:pd.DataFrame,y_dataFrame:pd.DataFrame,direction:int):
    x_dataFrame_direction = x_dataFrame[x_dataFrame['direction']==direction]
    y_dataFrame_direction = y_dataFrame[y_dataFrame['direction']==direction]

    if len(set(x_dataFrame_direction['date'])) != len(set(y_dataFrame_direction['date'])):
        same_dates = [date for date in set(y_dataFrame_direction['date']) if (date in set(x_dataFrame_direction['date']))]
        x_dataFrame_direction = x_dataFrame_direction[x_dataFrame_direction['date'].isin(same_dates)]
        y_dataFrame_direction = y_dataFrame_direction[y_dataFrame_direction['date'].isin(same_dates)]

    same_nums = [num for num in set(y_dataFrame_direction['num']) if (num in set(x_dataFrame_direction['num']))]
    x_target_dataFrame = x_dataFrame_direction[x_dataFrame_direction['num'].isin(same_nums)]
    x_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    x_target_dataFrame.reset_index(inplace=True)
    y_target_dataFrame = y_dataFrame_direction[y_dataFrame_direction['num'].isin(same_nums)]
    y_target_dataFrame.sort_values(by=['date','num'], inplace=True)
    y_target_dataFrame.reset_index(inplace=True)

    # 獲取 delay不存在的 row index
    null_index = list(x_target_dataFrame[x_target_dataFrame['delay'].isnull()].index)
    null_index.extend(list(y_target_dataFrame[y_target_dataFrame['delay'].isnull()].index))
    null_index = set(null_index)

    # 將delay不存在的 row 從資料中移除
    x_target_dataFrame = x_target_dataFrame.drop(index=null_index)
    y_target_dataFrame = y_target_dataFrame.drop(index=null_index)

    x = x_target_dataFrame['delay'].to_numpy()
    y = y_target_dataFrame['delay'].to_numpy()
    return x,y

In [189]:
def getRMSE(x:np.ndarray, y:np.ndarray):
    x = x.reshape(-1,1)
    xgbrModel=xgb.XGBRegressor()
    xgbrModel.fit(x,y)
    predicted=xgbrModel.predict(x)
    v = mean_squared_error(y,predicted)
    return sqrt(v)

In [190]:
dataFrame = pd.read_csv('/content/drive/MyDrive/railway.csv')
label_map = {'Keelung':1,'Xizhi':2,'Taipei':3,'Banqiao':4,'Taoyuan':5,'Zhongli':6,'Hsinchu':7,'Zhunan':8,'Miaoli':9,'Fengyuan':10,'Taichung':11,'Changhua':12,'Yuanlin':13,
'Douliu':14,'Chiayi':15,'Tainan':16,'Xinzuoying':17,'Kaohsiung':18,'Tanzi':0,'Taiyuan':0,'Xinwuri':0}

In [191]:
dataFrame_localTrain = dataFrame[dataFrame['num']<500]
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]>='2021/01/22']
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/02"]
dataFrame_localTrain = dataFrame_localTrain[dataFrame_localTrain["date"]!="2021/04/03"]

**連假日**

In [192]:
listlist = ["2021/02/10","2021/02/11","2021/02/12","2021/02/13","2021/02/14","2021/02/15","2021/02/16","2021/04/02","2021/04/03","2021/04/04","2021/04/05","2021/02/26","2021/02/27","2021/02/28","2021/03/01"]

In [193]:
dataFrame_holiday = dataFrame_localTrain[dataFrame_localTrain["date"].isin(listlist)]

In [194]:
dataFrame_holiday_Taipei = dataFrame_holiday[dataFrame_holiday['Station']=='Taipei']
dataFrame_holiday_Banqiao = dataFrame_holiday[dataFrame_holiday['Station']=="Banqiao"]
dataFrame_holiday_Taoyuan = dataFrame_holiday[dataFrame_holiday['Station']=='Taoyuan']
dataFrame_holiday_Zhongli = dataFrame_holiday[dataFrame_holiday['Station']=='Zhongli']
dataFrame_holiday_Hsinchu = dataFrame_holiday[dataFrame_holiday['Station']=='Hsinchu']
dataFrame_holiday_Zhunan = dataFrame_holiday[dataFrame_holiday['Station']=='Zhunan']
dataFrame_holiday_Miaoli = dataFrame_holiday[dataFrame_holiday['Station']=='Miaoli']

In [195]:
dataFrameList = [dataFrame_holiday_Taipei, dataFrame_holiday_Banqiao, dataFrame_holiday_Taoyuan, dataFrame_holiday_Zhongli, dataFrame_holiday_Hsinchu, dataFrame_holiday_Zhunan, dataFrame_holiday_Miaoli]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')
stationList = ['Taipei','Banqiao','Taoyuan','Zhongli','Hsinchu','Zhunan','Miaoli']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/i

[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/i

[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

In [196]:
mse_df.to_csv("連假日MSE.csv")
mse_df

,Taipei,Banqiao,Taoyuan,Zhongli,Hsinchu,Zhunan,Miaoli
Taipei,--,0.813756348003142,2.11795681390963,1.65699014846445,2.93822694191338,1.41275329007896,1.88935284569894
Banqiao,1.59294220906252,--,2.12087308710033,1.43869461163876,3.02456419637092,1.27864938159451,1.82531389233409
Taoyuan,2.44126779928914,1.54595533554095,--,0.696096209120518,2.52835078138440,1.19234710242389,1.66299502553648
Zhongli,2.44447562698856,1.65843403714138,0.780956717953596,--,2.65975281838727,1.34675817315110,1.61683508726877
Hsinchu,2.61245530494628,2.15819028184765,1.55632329948122,1.31154646803232,--,0.952129783638416,2.44455348639236
Zhunan,3.01635808022925,2.36961958656322,1.99770022350964,1.82632410248198,1.27112128660459,--,1.04062035030686
Miaoli,2.97794238577545,2.39693087166098,1.90985212644416,1.76049419929481,1.40078021855863,0.609283753716386,--


**全部**

In [197]:
dataFrame_localTrain_Taipei = dataFrame_localTrain[dataFrame_localTrain['Station']=='Taipei']
dataFrame_localTrain_Banqiao = dataFrame_localTrain[dataFrame_localTrain['Station']=="Banqiao"]
dataFrame_localTrain_Taoyuan = dataFrame_localTrain[dataFrame_localTrain['Station']=='Taoyuan']
dataFrame_localTrain_Zhongli = dataFrame_localTrain[dataFrame_localTrain['Station']=='Zhongli']
dataFrame_localTrain_Hsinchu = dataFrame_localTrain[dataFrame_localTrain['Station']=='Hsinchu']
dataFrame_localTrain_Zhunan = dataFrame_localTrain[dataFrame_localTrain['Station']=='Zhunan']
dataFrame_localTrain_Miaoli = dataFrame_localTrain[dataFrame_localTrain['Station']=='Miaoli']

In [198]:
dataFrameList = [dataFrame_localTrain_Taipei, dataFrame_localTrain_Banqiao, dataFrame_localTrain_Taoyuan, dataFrame_localTrain_Zhongli, dataFrame_localTrain_Hsinchu, dataFrame_localTrain_Zhunan, dataFrame_localTrain_Miaoli]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


[08:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/l

[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [199]:
stationList = ['Taipei','Banqiao','Taoyuan','Zhongli','Hsinchu','Zhunan','Miaoli']

mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)
mse_df

,Taipei,Banqiao,Taoyuan,Zhongli,Hsinchu,Zhunan,Miaoli
Taipei,--,0.739753332663508,1.45250577755842,1.41166867406890,1.89540061417789,1.64409563245432,1.81443008924391
Banqiao,1.25430661846289,--,1.33046253575400,1.26298496249101,1.90224591557204,1.55010323699135,1.73223662626678
Taoyuan,2.05737005073283,1.54931683643998,--,0.713482821059016,1.69942963680663,1.31891586863683,1.54334733765945
Zhongli,2.12828387125558,1.70006313906911,0.828397103071485,--,1.77237411563047,1.35721281435483,1.54840129851963
Hsinchu,3.23054182781578,3.10597783989393,2.81945308328947,2.69795728461867,--,0.985712837705084,1.83206359974947
Zhunan,2.51903827667580,2.27542499497180,1.83376175568952,1.59948651022747,0.978714801086018,--,1.10511495037548
Miaoli,3.61230239484982,3.47688664572680,3.26535067556445,3.13555215595429,1.19944279299259,0.649325230367047,--


In [200]:
dataFrame_localTrain_weekday = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() < 6)]
dataFrame_localTrain_weekday

,Station,direction,id,date,time,num,dest,delay,type
12,Zhongli,0,6,2021/01/29,07:07,103,Chaozhou,1.0,B
195,Zhongli,0,6,2021/01/22,07:07,103,Chaozhou,1.0,B
199,Zhongli,0,10,2021/01/22,07:44,105,Pingtung,1.0,B
206,Zhongli,0,16,2021/01/22,08:54,109,Changhua,0.0,B
208,Zhongli,0,18,2021/01/22,09:09,113,Pingtung,3.0,B
...,...,...,...,...,...,...,...,...,...
321004,Kaohsiung,1,82,2021/04/06,20:00,324,Tainan,3.0,B
321008,Kaohsiung,1,86,2021/04/06,20:53,326,Xinzuoying,0.0,A
321009,Kaohsiung,1,87,2021/04/06,21:09,156,Chiayi,0.0,B
321012,Kaohsiung,1,90,2021/04/06,21:58,328,Xinzuoying,1.0,B


In [201]:
dataFrame_localTrain_weekend = dataFrame_localTrain[dataFrame_localTrain['date'].apply(lambda date: datetime.date(int(date[:4]), int(date[5:7]), int(date[8:])).isoweekday() > 5)]
dataFrame_localTrain_weekend

,Station,direction,id,date,time,num,dest,delay,type
291,Zhongli,0,6,2021/01/23,07:07,103,Chaozhou,2.0,B
295,Zhongli,0,10,2021/01/23,07:44,105,Pingtung,2.0,B
300,Zhongli,0,15,2021/01/23,08:54,109,Changhua,1.0,B
302,Zhongli,0,17,2021/01/23,09:09,113,Pingtung,1.0,B
306,Zhongli,0,21,2021/01/23,09:44,115,Pingtung,4.0,B
...,...,...,...,...,...,...,...,...,...
320812,Kaohsiung,1,84,2021/04/04,20:00,324,Tainan,7.0,B
320816,Kaohsiung,1,88,2021/04/04,20:53,326,Xinzuoying,0.0,A
320817,Kaohsiung,1,89,2021/04/04,21:09,156,Chiayi,0.0,B
320820,Kaohsiung,1,92,2021/04/04,21:58,328,Xinzuoying,1.0,B


**假日**

In [202]:
dataFrame_localTrain_weekend_Taipei = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Taipei']
dataFrame_localTrain_weekend_Banqiao = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Banqiao']
dataFrame_localTrain_weekend_Taoyuan = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Taoyuan']
dataFrame_localTrain_weekend_Zhongli = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Zhongli']
dataFrame_localTrain_weekend_Hsinchu = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Hsinchu']
dataFrame_localTrain_weekend_Zhunan = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Zhunan']
dataFrame_localTrain_weekend_Miaoli = dataFrame_localTrain_weekend[dataFrame_localTrain_weekend['Station']=='Miaoli']

In [203]:
dataFrameList = [dataFrame_localTrain_Taipei, dataFrame_localTrain_Banqiao, dataFrame_localTrain_Taoyuan, dataFrame_localTrain_Zhongli, dataFrame_localTrain_Hsinchu, dataFrame_localTrain_Zhunan, dataFrame_localTrain_Miaoli]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Taipei','Banqiao','Taoyuan','Zhongli','Hsinchu','Zhunan','Miaoli']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


[08:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


[08:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernel

[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/l

In [204]:
mse_df.to_csv("假日MSE.csv")
mse_df

,Taipei,Banqiao,Taoyuan,Zhongli,Hsinchu,Zhunan,Miaoli
Taipei,--,0.739753332663508,1.45250577755842,1.41166867406890,1.89540061417789,1.64409563245432,1.81443008924391
Banqiao,1.25430661846289,--,1.33046253575400,1.26298496249101,1.90224591557204,1.55010323699135,1.73223662626678
Taoyuan,2.05737005073283,1.54931683643998,--,0.713482821059016,1.69942963680663,1.31891586863683,1.54334733765945
Zhongli,2.12828387125558,1.70006313906911,0.828397103071485,--,1.77237411563047,1.35721281435483,1.54840129851963
Hsinchu,3.23054182781578,3.10597783989393,2.81945308328947,2.69795728461867,--,0.985712837705084,1.83206359974947
Zhunan,2.51903827667580,2.27542499497180,1.83376175568952,1.59948651022747,0.978714801086018,--,1.10511495037548
Miaoli,3.61230239484982,3.47688664572680,3.26535067556445,3.13555215595429,1.19944279299259,0.649325230367047,--


**平日**

In [205]:
dataFrame_localTrain_weekday_Taipei = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Taipei']
dataFrame_localTrain_weekday_Banqiao = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Banqiao']
dataFrame_localTrain_weekday_Taoyuan = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Taoyuan']
dataFrame_localTrain_weekday_Zhongli = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Zhongli']
dataFrame_localTrain_weekday_Hsinchu = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Hsinchu']
dataFrame_localTrain_weekday_Zhunan = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Zhunan']
dataFrame_localTrain_weekday_Miaoli = dataFrame_localTrain_weekday[dataFrame_localTrain_weekday['Station']=='Miaoli']

In [206]:
dataFrameList = [dataFrame_localTrain_weekday_Taipei, dataFrame_localTrain_weekday_Banqiao, dataFrame_localTrain_weekday_Taoyuan, dataFrame_localTrain_weekday_Zhongli, dataFrame_localTrain_weekday_Hsinchu, dataFrame_localTrain_weekday_Zhunan, dataFrame_localTrain_weekday_Miaoli]
mseList = []
for positionStart in range(len(dataFrameList)):
    mseList.append([])
    dataFrameStart = dataFrameList[positionStart]
    for positionEnd in range(len(dataFrameList)):
        dataFrameEnd = dataFrameList[positionEnd]
        if positionStart > positionEnd:  # 順行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 0)
            mseList[positionStart].append(getRMSE(x,y))
        elif positionStart < positionEnd: # 逆行
            x,y = arrangeDataFrame(dataFrameStart, dataFrameEnd, 1)
            mseList[positionStart].append(getRMSE(y,x))
        else:
            mseList[positionStart].append('--')

stationList = ['Taipei','Banqiao','Taoyuan','Zhongli','Hsinchu','Zhunan','Miaoli']
mseDict = dict(zip(stationList, mseList))
mse_df = pd.DataFrame(mseDict, index=stationList)

[08:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/i

[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.

[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/l

[08:09:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:09:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.

In [207]:
mse_df.to_csv("平日MSE.csv")
mse_df

,Taipei,Banqiao,Taoyuan,Zhongli,Hsinchu,Zhunan,Miaoli
Taipei,--,0.743299770984658,1.35835159577891,1.33887691757911,1.67350047065223,1.58549487779681,1.73217822647333
Banqiao,1.14637274232872,--,1.21363763309589,1.19919448393874,1.68365656577529,1.55237119771315,1.68483584503679
Taoyuan,1.84825901745144,1.48682245791560,--,0.700653987046745,1.51181859287030,1.32700922219461,1.56135239513999
Zhongli,1.93111096930704,1.61969858434890,0.805508726672410,--,1.56651432897067,1.34790245236089,1.57182434410029
Hsinchu,3.41711649575994,3.40135774296797,3.18276233579599,3.09106567258036,--,0.944957206348117,1.71568865740907
Zhunan,2.37422135908595,2.22728752950699,1.85425759789452,1.64152473511393,1.01747726637854,--,1.09105626403075
Miaoli,3.83263797766795,3.78854432845808,3.65215809990782,3.53429783023724,1.18978877544059,0.626662334154259,--
